# Text generation with LSTM

В этом семинаре мы научимся генерировать тексты с помощью нейронных сетей.

Сначала просто посмотрите код, ***не запуская его***, чтобы понять алгоритм генерации текстов.


Загрузим текст из файла:

In [1]:
raw_text = ""
with open('friends.txt', 'r') as f:
  for line in f.read().split('\n'):
    raw_text += line.lower()
    
print(len(raw_text))
print(raw_text[:100])

72798
tv show episode scripts  >  friends  >  season 1  >  the one where it all beganfriends s01e01 episod


Т.к. генерировать текст мы будем побуквенно, составим словарь из всех знаков, которые имеются у нас в тексте:

In [2]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

seq_length = 50

Total Characters:  72798
Total Vocab:  50


Напишем функцию, которая векторизует текст: на вход поступает строка, каждый символ строки кодируется как one-hot вектор:

In [3]:
import numpy as np

def vectorize_string(string):
  vector = np.zeros((seq_length, n_vocab))
  for i, char in enumerate(string):
    v = np.zeros(n_vocab)
    v[char_to_int[char]] = 1.0
    vector[i] = v
  return vector
    

Векторизуем весь датасет. Каждый сэмпл должен иметь вид:

`последовательность символов -> следующий символ этой последовательности`

Другими словами, признаками, которые мы подаём на вход сети, является последовательность; класс, который нужно предсказать соответствует одному из символов в нашем словаре.

In [4]:

dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  if i < 10:
    print(seq_in, "=>", seq_out)
  dataX.append(vectorize_string(seq_in))
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

tv show episode scripts  >  friends  >  season 1   => >
v show episode scripts  >  friends  >  season 1  > =>  
 show episode scripts  >  friends  >  season 1  >  =>  
show episode scripts  >  friends  >  season 1  >   => t
how episode scripts  >  friends  >  season 1  >  t => h
ow episode scripts  >  friends  >  season 1  >  th => e
w episode scripts  >  friends  >  season 1  >  the =>  
 episode scripts  >  friends  >  season 1  >  the  => o
episode scripts  >  friends  >  season 1  >  the o => n
pisode scripts  >  friends  >  season 1  >  the on => e
Total Patterns:  72748


In [5]:
import numpy as np
from keras.utils import np_utils

X = np.reshape(dataX, (n_patterns, seq_length, n_vocab))
y = np_utils.to_categorical(dataY)

print(X.shape)
print(y.shape)


(72748, 50, 50)
(72748, 50)


Наша модель:

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 50, 256)           314368    
_________________________________________________________________
dropout_4 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                12850     
Total params: 852,530
Trainable params: 852,530
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
stopper = EarlyStopping(monitor='loss', patience=3, verbose=1, mode='auto')

model.fit(X, y, epochs=10, batch_size=64, callbacks=[stopper])

Epoch 1/10
1137/1137 [==============================] - 13s 12ms/step - loss: 2.5401
Epoch 2/10
1137/1137 [==============================] - 13s 12ms/step - loss: 2.0090
Epoch 3/10
1137/1137 [==============================] - 13s 12ms/step - loss: 1.8036
Epoch 4/10
1137/1137 [==============================] - 13s 12ms/step - loss: 1.6751
Epoch 5/10
1137/1137 [==============================] - 13s 12ms/step - loss: 1.5839
Epoch 6/10
1137/1137 [==============================] - 13s 12ms/step - loss: 1.5029
Epoch 7/10
1137/1137 [==============================] - 13s 12ms/step - loss: 1.4317
Epoch 8/10
1137/1137 [==============================] - 13s 12ms/step - loss: 1.3656
Epoch 9/10
1137/1137 [==============================] - 13s 12ms/step - loss: 1.3046
Epoch 10/10
1137/1137 [==============================] - 13s 12ms/step - loss: 1.2425


После того, как модель обучилась, попробуем сгенерировать текст. Для этого мы должны дать модели на вход какой-нибудь начальный текст и определить, сколько символов модель будет генерировать (в данном примере 100):

In [14]:
import sys

beginning = "i love "

ending = ""

for i in range(100):
  text = beginning + ending
  if len(text) > seq_length:
    text = text[len(text)-seq_length:]
  else:
    text = text.rjust(seq_length, ' ')
    
  vector = vectorize_string(text)
  vector = [vector]
  vector = np.array(vector)
  
  
  prediction = model.predict(vector, verbose=0)
  index = np.argmax(prediction)
  result = int_to_char[index]
  sys.stdout.write(result)
  ending += result
  

in a play! - what if i don't want to be a little.- i didn't know you got this for the comedian to th

После того, как вы посмотрели на пример, настало время обучить свою модель!

Найдите текст, на котором будет обучаться ваша модель. Желательно, чтобы он имел какие-то отличительные особенности (например, политические тексты/рецепты/определённого автора/пр.).

Объём текста должен быть достаточно большим - около 100 000 последовательностей, на которых будет обучаться модель.

После того, как вы собрали тексты - загрузите его, замените имя файла в первой ячейке и запускайте код:)